# Tutorial 6: Agent Configuration and Memory Management

**Difficulty:** Intermediate | **Time:** 30 minutes

## Learning Objectives

- Master AgentConfig for structured agent configuration
- Understand and implement different memory policies
- Configure memory windows and conversation history
- Work with preloaded messages for context initialization
- Distinguish between agent state and memory
- Implement stateless vs stateful agents

## Real-World Use Case

Imagine you're building AI assistants for different scenarios:

- **Customer Support Bot**: Needs to remember the entire conversation to provide context-aware help
- **FAQ Assistant**: Stateless, answers each question independently for efficiency
- **Therapy Chatbot**: Maintains a rolling window of recent conversation to stay relevant without overwhelming the model
- **Code Review Agent**: Preloaded with project context and coding standards
- **Research Assistant**: Long-term memory with summarization of previous research sessions

Each of these requires different **memory management strategies** and **configuration patterns**. In this tutorial, you'll learn how to configure agents for various memory requirements and understand the tradeoffs between different approaches.

## Core Concepts

### AgentConfig: Centralized Configuration

**AgentConfig** is a Pydantic model that centralizes all agent configuration in one place:

```python
config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-mini'),
    system_prompt='You are a helpful assistant.',
    prompt_template='User question: {{ query }}',
    memory_config=MemoryConfig(policy=MemoryPolicyType.ROLLING_WINDOW, window=10),
    preload_messages=[...],
    output_mode='text',
    tools=[...],
)
agent = Agent(config)
```

**Benefits:**
- Type-safe configuration with Pydantic validation
- Easy to serialize/deserialize for saving agent configurations
- Clear separation between configuration and runtime behavior
- Reusable across multiple agent instances

### Memory vs. State vs. GraphState: Critical Distinctions

**Memory** (Conversation History):
- Stores the **message history** sent to the LLM
- Managed by `MemoryManager` via `memory_config`
- Controls what the LLM "sees" from past interactions
- Policy-driven (NULL, ROLLING_WINDOW, CUSTOM)
- Impacts token usage and context window
- **Agent-level scope**

**State** (Node State):
- Stores **application-level data** like counters, flags, metadata
- Node-local, does NOT flow between nodes
- Persists across node invocations (e.g., in loops)
- Accessed via `context.state`
- Does NOT affect LLM input
- **Node-level scope**

**GraphState** (Global Shared State):
- **Shared across all nodes** in a graph
- Thread-safe with automatic locking for concurrent execution
- Accessed via `context.graph_state`
- Enables coordination between multiple agents/nodes
- Does NOT affect LLM input (unless explicitly passed)
- **Graph-level scope**
- Perfect for multi-agent workflows and global tracking

**Example:**
```python
# Memory: What the LLM sees (agent-level)
agent.memory = [
    {'role': 'user', 'content': 'Hello!'},
    {'role': 'assistant', 'content': 'Hi! How can I help?'},
]

# State: Node-local tracking (NOT sent to LLM)
context.state = {
    'conversation_id': 'conv_123',
    'user_id': 'user_456',
    'message_count': 2,
}

# GraphState: Shared across all nodes (NOT sent to LLM unless passed explicitly)
await context.graph_state.set('total_messages', 150)
await context.graph_state.set('active_agents', ['agent1', 'agent2'])
counter = await context.graph_state.get('workflow_iteration', 0)
```

### Memory Policies

Spark provides three memory policies:

**1. NULL Policy** (Stateless)
- No conversation history is maintained
- Each message is processed independently
- Most efficient for token usage
- Best for: FAQ bots, simple Q&A, stateless APIs

**2. ROLLING_WINDOW Policy** (Bounded Memory)
- Keeps only the last N messages
- Automatically drops oldest messages when limit is reached
- Balances context and token efficiency
- Best for: Chat bots, support agents, conversational interfaces

**3. CUSTOM Policy** (Advanced)
- Use a custom callable to manage memory
- Full control over message retention logic
- Can implement summarization, importance scoring, etc.
- Best for: Complex memory requirements, research assistants, adaptive agents

**Note:** The tutorial plan mentioned a SUMMARIZE policy which would automatically summarize old messages to compress context. This is not yet implemented in Spark but can be achieved via CUSTOM policy.

### Memory Configuration

```python
from spark.agents.memory import MemoryConfig, MemoryPolicyType

# Stateless agent (no memory)
memory_config = MemoryConfig(policy=MemoryPolicyType.NULL)

# Rolling window with 10 messages
memory_config = MemoryConfig(
    policy=MemoryPolicyType.ROLLING_WINDOW,
    window=10
)

# Custom memory management
def custom_memory_fn(messages: list[dict]) -> list[dict]:
    # Keep only high-priority messages
    return [m for m in messages if m.get('priority') == 'high']

memory_config = MemoryConfig(
    policy=MemoryPolicyType.CUSTOM,
    callable=custom_memory_fn
)
```

## Setup

Let's import the necessary classes for agent configuration and memory management.

In [1]:
# Import core Spark classes
from spark.nodes import Node, ExecutionContext, NodeState, default_node_state
from spark.agents.agent import Agent
from spark.agents.config import AgentConfig
from spark.agents.memory import MemoryConfig, MemoryPolicyType
from spark.tools import tool
from spark.models.openai import OpenAIModel
from spark.models.echo import EchoModel
from spark.utils import arun
from typing import Any
import time

## Example 1: Basic AgentConfig - The Clean Way

Let's start by comparing the old way vs. the AgentConfig way.

In [2]:
config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-mini'),
    name='math_tutor',
    description='A helpful math tutor that explains concepts clearly',
    system_prompt='You are a helpful math tutor.',
    prompt_template='Help me solve: {{ problem }}',
    output_mode='text',
)
new_agent = Agent(config)
print("✓ Agent created with clean, centralized configuration")
print(f"✓ Agent name: {new_agent.config.name}")
print(f"✓ Agent description: {new_agent.config.description}")
print()

# Test the agent
result = await new_agent.run({'problem': 'What is 15 + 27?'})
print(f"\n📝 Agent response: {result.content}")

✓ Agent created with clean, centralized configuration
✓ Agent name: math_tutor
✓ Agent description: A helpful math tutor that explains concepts clearly


📝 Agent response: 15 + 27 = 42

(10 + 20 = 30, 5 + 7 = 12, and 30 + 12 = 42.)


### Understanding AgentConfig Benefits

**Why use AgentConfig?**

1. **Centralized Configuration**: All agent settings in one place
2. **Type Safety**: Pydantic validation catches errors early
3. **Reusability**: Save and load configurations
4. **Serialization**: Easy to export to JSON for specs
5. **Documentation**: Clear API for what can be configured
6. **Testing**: Easy to create test configurations

**Key Configuration Options:**
- `model`: The LLM model to use
- `name`: Identifier for the agent (useful in multi-agent systems)
- `description`: What the agent does (for documentation)
- `system_prompt`: System instructions for the LLM
- `prompt_template`: Jinja2 template for formatting user inputs
- `memory_config`: Memory management settings
- `preload_messages`: Initial conversation history
- `output_mode`: 'text' or 'json' (structured output)
- `output_schema`: Pydantic schema for structured output
- `tools`: List of tools available to the agent
- `callback_handler`: Event handling during execution
- `hooks`: Pre/post processing hooks

## Example 2: Stateless Agent with NULL Memory Policy

A stateless agent doesn't remember previous interactions. Each message is processed independently.

In [3]:
print("=== Example 2: Stateless Agent (NULL Memory) ===")
print()

# Create a stateless FAQ agent
faq_config = AgentConfig(
    model=EchoModel(),
    name='faq_bot',
    system_prompt='You are a FAQ bot. Answer questions based solely on the current question.',
    memory_config=MemoryConfig(policy=MemoryPolicyType.NULL),
)
faq_agent = Agent(faq_config)

print("🤖 FAQ Bot (Stateless - No Memory)")
print(f"Memory Policy: {faq_agent.config.memory_config.policy}")
print()

# Simulate a conversation
questions = [
    "What are your business hours?",
    "Do you offer refunds?",
    "What did I just ask you?",  # Agent won't remember!
]

for i, question in enumerate(questions, 1):
    print(f"Q{i}: {question}")
    # Use structured content format
    result = await faq_agent.do({'messages': [{'role': 'user', 'content': [{'text': question}]}]})
    print(f"A{i}: {result.content}")
    print()

print("📊 Analysis:")
print("✓ Question 3 asks about previous question, but agent has no memory")
print("✓ Each query is independent - most efficient for simple Q&A")
print("✓ No token overhead from conversation history")
print("✓ Perfect for: FAQ bots, stateless APIs, simple lookups")

=== Example 2: Stateless Agent (NULL Memory) ===

🤖 FAQ Bot (Stateless - No Memory)
Memory Policy: null

Q1: What are your business hours?
A1: What are your business hours?

Q2: Do you offer refunds?
A2: Do you offer refunds?

Q3: What did I just ask you?
A3: What did I just ask you?

📊 Analysis:
✓ Question 3 asks about previous question, but agent has no memory
✓ Each query is independent - most efficient for simple Q&A
✓ No token overhead from conversation history
✓ Perfect for: FAQ bots, stateless APIs, simple lookups


### Understanding NULL Memory Policy

**When to use NULL policy:**
- FAQ bots where context isn't needed
- API endpoints that process independent requests
- High-volume services where memory overhead is problematic
- Classification or single-shot inference tasks

**Characteristics:**
- ✅ Most token-efficient (no history sent to LLM)
- ✅ Fastest response times
- ✅ Stateless and horizontally scalable
- ❌ No conversation continuity
- ❌ Cannot reference previous messages
- ❌ Poor user experience for conversational use cases

**Best Practices:**
- Use for simple question-answering
- Include all necessary context in each request
- Set clear expectations with users ("I don't remember previous messages")
- Consider adding context in system prompt if needed

## Example 3: Conversational Agent with Rolling Window Memory

A rolling window keeps the most recent N messages, providing context without unbounded growth.

In [4]:
print("=== Example 3: Conversational Agent (Rolling Window Memory) ===")
print()

# Create an agent with rolling window memory
chat_config = AgentConfig(
    model=EchoModel(),
    name='chat_assistant',
    system_prompt='You are a friendly chat assistant. Remember the conversation context.',
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=4  # Keep last 4 messages (2 turns)
    ),
)
chat_agent = Agent(chat_config)

print("🤖 Chat Assistant (Rolling Window Memory)")
print(f"Memory Policy: {chat_agent.config.memory_config.policy}")
print(f"Memory Window: {chat_agent.config.memory_config.window} messages")
print()

# Simulate a conversation
conversation = [
    "Hi! My name is Alice.",
    "I'm working on a Python project.",
    "It's a web application using Flask.",
    "What was my name again?",  # Should remember
    "And what framework am I using?",  # Should remember
    "What was the first thing I told you?",  # Might not remember (outside window)
]

# Note: In the current implementation, we need to manually manage memory
# This is a demonstration of how it should work
print("💬 Conversation:")
for i, message in enumerate(conversation, 1):
    print(f"\nTurn {i}:")
    print(f"  User: {message}")
    
    # Use structured content format
    result = await chat_agent.do({'messages': [{'role': 'user', 'content': [{'text': message}]}]})
    print(f"  Assistant: {result.content}")
    
    # Demonstrate memory window
    if i == 4:
        print("  📝 Memory: Should remember Alice (within window)")
    elif i == 5:
        print("  📝 Memory: Should remember Flask (within window)")
    elif i == 6:
        print("  📝 Memory: First message may be outside window now")

print()
print("📊 Analysis:")
print("✓ Rolling window maintains recent context")
print("✓ Old messages are automatically dropped")
print("✓ Balances context awareness with token efficiency")
print("✓ Perfect for: Chat bots, support agents, conversational interfaces")

=== Example 3: Conversational Agent (Rolling Window Memory) ===

🤖 Chat Assistant (Rolling Window Memory)
Memory Policy: rolling_window
Memory Window: 4 messages

💬 Conversation:

Turn 1:
  User: Hi! My name is Alice.
  Assistant: Hi! My name is Alice.

Turn 2:
  User: I'm working on a Python project.
  Assistant: Hi! My name is Alice.
Hi! My name is Alice.
I'm working on a Python project.

Turn 3:
  User: It's a web application using Flask.
  Assistant: Hi! My name is Alice.
Hi! My name is Alice.
I'm working on a Python project.
Hi! My name is Alice.
Hi! My name is Alice.
I'm working on a Python project.
It's a web application using Flask.

Turn 4:
  User: What was my name again?
  Assistant: I'm working on a Python project.
Hi! My name is Alice.
Hi! My name is Alice.
I'm working on a Python project.
It's a web application using Flask.
Hi! My name is Alice.
Hi! My name is Alice.
I'm working on a Python project.
Hi! My name is Alice.
Hi! My name is Alice.
I'm working on a Python projec

### Understanding Rolling Window Memory

**How it works:**
1. New messages are added to memory
2. When memory exceeds window size, oldest message is removed (FIFO)
3. Only messages in window are sent to LLM

**When to use Rolling Window:**
- Conversational agents that need recent context
- Chat bots with ongoing conversations
- Support agents handling customer inquiries
- Any scenario where context is needed but not entire history

**Characteristics:**
- ✅ Maintains recent conversation context
- ✅ Bounded token usage (predictable costs)
- ✅ Simple to configure and understand
- ✅ Good balance of context and efficiency
- ❌ Loses older conversation details
- ❌ May forget important info mentioned early
- ⚠️ Need to tune window size for use case

**Choosing Window Size:**
- **Small (2-4 messages)**: Quick exchanges, simple tasks
- **Medium (6-10 messages)**: Standard conversations, most chat bots
- **Large (12-20 messages)**: Complex discussions, support sessions
- **Very Large (20+ messages)**: Consider CUSTOM policy with summarization

**Pro Tip:** Monitor your token usage and adjust window size based on:
- Average message length
- Model context window limits
- Cost constraints
- Quality of responses (bigger window = better context)

## Example 4: Preloading Messages - Context Initialization

Preload messages to initialize an agent with specific context, examples, or conversation history.

In [5]:
print("=== Example 4: Preloading Messages ===")
print()

# Create an agent with preloaded conversation
preloaded_messages = [
    {'role': 'user', 'content': [{'text':'What is your coding style?'}]},
    {'role': 'assistant', 'content': [{'text': 'I follow PEP 8 for Python, use descriptive variable names, and prefer functional programming patterns.'}]},
    {'role': 'user', 'content': [{'text': 'What about documentation?'}]},
    {'role': 'assistant', 'content': [{'text': 'I always write docstrings for functions and classes, and include type hints for clarity.'}]},
]

code_reviewer_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='code_reviewer',
    system_prompt='You are a code reviewer who has established coding standards.',
    preload_messages=preloaded_messages,
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=10
    ),
)
code_reviewer = Agent(code_reviewer_config)

print("🤖 Code Reviewer (With Preloaded Context)")
print(f"Preloaded messages: {len(preloaded_messages)}")
print()
print("📜 Preloaded conversation:")
for msg in preloaded_messages:
    role = msg['role'].capitalize()
    # Extract text from content blocks
    text_content = msg['content'][0]['text'] if isinstance(msg['content'], list) else msg['content']
    content = text_content[:60] + '...' if len(text_content) > 60 else text_content
    print(f"  {role}: {content}")
print()

# Now ask a question that builds on preloaded context
print("💬 New conversation:")
question = "Can you review this function? It doesn't have a docstring."
print(f"User: {question}")

# Use structured content format
result = await code_reviewer.do({'messages': [{'role': 'user', 'content': [{'text': question}]}]})
print(f"Assistant: {result.content}")
print()

print("📊 Analysis:")
print("✓ Agent starts with established context")
print("✓ No need to re-establish standards each session")
print("✓ Saves tokens by not repeating setup information")
print("✓ Perfect for: Domain experts, specialized assistants, code reviewers")

=== Example 4: Preloading Messages ===

🤖 Code Reviewer (With Preloaded Context)
Preloaded messages: 4

📜 Preloaded conversation:
  User: What is your coding style?
  Assistant: I follow PEP 8 for Python, use descriptive variable names, a...
  User: What about documentation?
  Assistant: I always write docstrings for functions and classes, and inc...

💬 New conversation:
User: Can you review this function? It doesn't have a docstring.
Assistant: I can review it, but I don’t see the function yet. Please paste the function (or a link) and I’ll critique the absence/presence of a docstring and suggest a concrete docstring.

In the meantime, here’s how I’d review a function that’s missing a docstring, plus ready-to-use templates you can drop in.

Review checklist for a missing docstring
- Purpose: Is the function’s responsibility clear from the code? If not, a brief summary should be included in the docstring.
- Summary line: Is there a concise one-line summary at the top?
- Parameters (Arg

### Understanding Preloaded Messages

**Use cases for preloading:**
1. **Few-shot examples**: Show the agent how to respond
2. **Context initialization**: Establish domain knowledge
3. **Persona establishment**: Define agent's character/style
4. **Project context**: Load project-specific information
5. **Conversation resumption**: Continue a previous session

**Benefits:**
- ✅ Consistent behavior across sessions
- ✅ No need to repeat context setup
- ✅ Can include few-shot examples
- ✅ Works with any memory policy

**Best Practices:**
- Keep preloaded messages concise (they count toward token limits)
- Use for establishing context, not for lengthy histories
- Combine with appropriate memory policy
- Consider whether preloaded messages should be in the rolling window

**Example Use Cases:**
```python
# Few-shot classification
preload_messages = [
    {'role': 'user', 'content': 'I love this product!'},
    {'role': 'assistant', 'content': 'Sentiment: Positive'},
    {'role': 'user', 'content': 'This is terrible.'},
    {'role': 'assistant', 'content': 'Sentiment: Negative'},
]

# Project context
preload_messages = [
    {'role': 'user', 'content': 'What stack are we using?'},
    {'role': 'assistant', 'content': 'Python FastAPI backend, React frontend, PostgreSQL database.'},
]
```

## Example 5: Custom Memory Policy - Advanced Control

For complex memory requirements, implement a custom memory policy with full control.

In [6]:
print("=== Example 5: Custom Memory Policy ===")
print()

# Define a custom memory function
def priority_based_memory(messages: list[dict[str, Any]]) -> list[dict[str, Any]]:
    """
    Custom memory policy that keeps:
    - All messages marked as 'important'
    - Last 5 regular messages
    """
    important_messages = [m for m in messages if m.get('metadata', {}).get('important', False)]
    regular_messages = [m for m in messages if not m.get('metadata', {}).get('important', False)]
    
    # Keep last 5 regular messages
    recent_regular = regular_messages[-5:]
    
    # Combine: important messages + recent regular messages
    result = important_messages + recent_regular
    
    print(f"  📊 Memory filtered: {len(messages)} → {len(result)} messages")
    print(f"     Important: {len(important_messages)}, Recent: {len(recent_regular)}")
    
    return result

# Create agent with custom memory policy
custom_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='smart_assistant',
    system_prompt='You are an assistant that prioritizes important information.',
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.CUSTOM,
        callable=priority_based_memory
    ),
)
smart_agent = Agent(custom_config)

print("🤖 Smart Assistant (Custom Memory Policy)")
print(f"Memory Policy: {smart_agent.config.memory_config.policy}")
print("Memory Strategy: Keep all important messages + last 5 regular messages")
print()

# Demonstrate custom memory behavior
print("💬 Conversation with priority tagging:")
print()

# In a real implementation, you would add messages with metadata
# This is a conceptual demonstration
messages_demo = [
    {'role': 'user', 'content': 'Set my budget to $5000', 'metadata': {'important': True}},
    {'role': 'assistant', 'content': 'Budget set to $5000'},
    {'role': 'user', 'content': 'What\'s the weather?'},
    {'role': 'assistant', 'content': 'Sunny today'},
    {'role': 'user', 'content': 'Tell me a joke'},
    {'role': 'assistant', 'content': 'Why did the chicken...'},
    {'role': 'user', 'content': 'My account number is 12345', 'metadata': {'important': True}},
    {'role': 'assistant', 'content': 'Account number saved'},
    # ... many more messages ...
]

print("Simulating memory filtering:")
filtered = priority_based_memory(messages_demo)
print()

print("📋 Retained messages:")
for msg in filtered:
    marker = "⭐" if msg.get('metadata', {}).get('important') else "💬"
    content = msg['content'][:40] + '...' if len(msg['content']) > 40 else msg['content']
    print(f"  {marker} {msg['role']}: {content}")

print()
print("📊 Analysis:")
print("✓ Important information (budget, account) always retained")
print("✓ Recent messages provide context")
print("✓ Old unimportant messages dropped")
print("✓ Perfect for: Long-running agents, research assistants, adaptive memory")

=== Example 5: Custom Memory Policy ===

🤖 Smart Assistant (Custom Memory Policy)
Memory Policy: custom
Memory Strategy: Keep all important messages + last 5 regular messages

💬 Conversation with priority tagging:

Simulating memory filtering:
  📊 Memory filtered: 8 → 7 messages
     Important: 2, Recent: 5

📋 Retained messages:
  ⭐ user: Set my budget to $5000
  ⭐ user: My account number is 12345
  💬 user: What's the weather?
  💬 assistant: Sunny today
  💬 user: Tell me a joke
  💬 assistant: Why did the chicken...
  💬 assistant: Account number saved

📊 Analysis:
✓ Important information (budget, account) always retained
✓ Recent messages provide context
✓ Old unimportant messages dropped
✓ Perfect for: Long-running agents, research assistants, adaptive memory


### Understanding Custom Memory Policy

**When to use CUSTOM policy:**
- Complex memory requirements
- Importance-based retention
- Summarization of old messages
- Topic-based memory management
- Semantic similarity filtering

**Custom Memory Patterns:**

**1. Importance-Based (shown above)**
- Keep messages marked as important
- Drop less relevant messages
- Useful for long-running assistants

**2. Summarization Pattern**
```python
def summarizing_memory(messages: list[dict]) -> list[dict]:
    if len(messages) > 20:
        # Summarize first 10 messages
        old_messages = messages[:10]
        summary = create_summary(old_messages)  # Use LLM to summarize
        recent_messages = messages[10:]
        return [{'role': 'system', 'content': f'Summary: {summary}'}] + recent_messages
    return messages
```

**3. Topic-Based Pattern**
```python
def topic_based_memory(messages: list[dict]) -> list[dict]:
    # Keep only messages related to current topic
    current_topic = extract_topic(messages[-1])
    return [m for m in messages if extract_topic(m) == current_topic]
```

**4. Semantic Similarity Pattern**
```python
def semantic_memory(messages: list[dict]) -> list[dict]:
    # Keep messages semantically similar to recent queries
    recent_query = messages[-1]
    embeddings = get_embeddings([m['content'] for m in messages])
    similarities = compute_similarity(embeddings[-1], embeddings)
    return [m for i, m in enumerate(messages) if similarities[i] > 0.7]
```

**Best Practices:**
- Keep custom functions efficient (called on every message)
- Always return a valid message list
- Consider token limits when retaining messages
- Log memory decisions for debugging
- Test with realistic conversation lengths

## Example 6: State, Memory, and GraphState in Action

Let's clearly demonstrate the differences between state, memory, and GraphState with a multi-agent scenario.

In [ ]:
print("=== Example 6: State, Memory, and GraphState ===")
print()

from spark.graphs import Graph

class CoordinatorAgent(Agent):
    """Agent that coordinates workflow and tracks node-level state."""
    
    async def process(self, context: ExecutionContext):
        # NODE STATE: Application-level tracking (NOT sent to LLM)
        message_count = context.state.get('message_count', 0) + 1
        context.state['message_count'] = message_count
        context.state['last_timestamp'] = time.time()
        
        print(f"📊 Node State (Coordinator - NOT sent to LLM):")
        print(f"   Message count (this node): {message_count}")
        
        # GRAPH STATE: Shared across all nodes
        if context.graph_state:
            total_messages = await context.graph_state.get('total_workflow_messages', 0)
            await context.graph_state.set('total_workflow_messages', total_messages + 1)
            
            print(f"\n🌐 Graph State (Shared - NOT sent to LLM):")
            print(f"   Total workflow messages (all nodes): {total_messages + 1}")
        
        # MEMORY: Conversation history (SENT to LLM)
        print(f"\n💬 Memory (Conversation History - SENT to LLM):")
        print(f"   Managed by MemoryManager, includes all messages")
        print()
        
        # Call parent process to handle LLM interaction
        result = await super().process(context)
        return result

class ProcessorAgent(Agent):
    """Another agent to show state isolation and GraphState sharing."""
    
    async def process(self, context: ExecutionContext):
        # This agent has its own node state
        process_count = context.state.get('process_count', 0) + 1
        context.state['process_count'] = process_count
        
        print(f"📊 Node State (Processor - NOT sent to LLM):")
        print(f"   Process count (this node): {process_count}")
        
        # But shares GraphState with Coordinator
        if context.graph_state:
            total_messages = await context.graph_state.get('total_workflow_messages', 0)
            await context.graph_state.set('total_workflow_messages', total_messages + 1)
            
            # Get coordinator's contribution
            print(f"\n🌐 Graph State (Shared - accessible by all nodes):")
            print(f"   Total workflow messages: {total_messages + 1}")
            print(f"   (Includes messages from Coordinator agent)")
        
        print()
        result = await super().process(context)
        return result

# Create agents with rolling window memory
coordinator_state = default_node_state()
coordinator_state['message_count'] = 0

processor_state = default_node_state()
processor_state['process_count'] = 0

coordinator_config = AgentConfig(
    model=EchoModel(),
    name='coordinator',
    system_prompt='You coordinate the workflow.',
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=3
    ),
    initial_state=coordinator_state
)

processor_config = AgentConfig(
    model=EchoModel(),
    name='processor',
    system_prompt='You process data.',
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=3
    ),
    initial_state=processor_state
)

coordinator = CoordinatorAgent(config=coordinator_config)
processor = ProcessorAgent(config=processor_config)

# Create a simple graph with both agents
coordinator >> processor

graph = Graph(
    start=coordinator,
    initial_state={'total_workflow_messages': 0}  # Initialize GraphState
)

print("🤖 Demonstrating State vs Memory vs GraphState")
print()
print("Scenario: Two agents in a workflow")
print("- Coordinator: Tracks its own message_count (node state)")
print("- Processor: Tracks its own process_count (node state)")
print("- Both: Share total_workflow_messages (graph state)")
print()

# Run workflow
print("--- Running workflow ---")
result = await graph.run({'messages': [{'role': 'user', 'content': [{'text': 'Start workflow'}]}]})

print("\n" + "="*60)
print("📚 Summary: State vs Memory vs GraphState")
print()
print("NODE STATE (context.state):")
print("  ✓ Node-local tracking (each agent has its own)")
print("  ✓ Does NOT flow between nodes")
print("  ✓ NOT sent to LLM")
print("  ✓ Used for: per-node counters, flags, metadata")
print()
print("MEMORY (agent.memory / MemoryManager):")
print("  ✓ Conversation history per agent")
print("  ✓ IS sent to LLM as context")
print("  ✓ Managed by memory_config policy")
print("  ✓ Agent-level scope")
print("  ✓ Used for: conversation context and continuity")
print()
print("GRAPH STATE (context.graph_state):")
print("  ✓ Shared across ALL nodes in the graph")
print("  ✓ Thread-safe with automatic locking")
print("  ✓ NOT sent to LLM (unless explicitly passed)")
print("  ✓ Graph-level scope")
print("  ✓ Used for: multi-agent coordination, global counters, workflow state")
print()
print("When to use each:")
print("  • Node State: Track per-agent/per-node metrics")
print("  • Memory: Enable LLM conversation continuity")
print("  • Graph State: Coordinate between multiple agents/nodes")

### Understanding State, Memory, and GraphState

| Aspect | Node State | Memory | GraphState |
|--------|-----------|--------|------------|
| **Purpose** | Per-node tracking | LLM conversation context | Cross-node coordination |
| **Sent to LLM?** | ❌ No | ✅ Yes | ❌ No (unless passed) |
| **Scope** | Node-local | Agent-level | Graph-level (all nodes) |
| **Management** | Manual in process() | Automatic via MemoryManager | Automatic with locking |
| **Sharing** | Not shared | Not shared | Shared across all nodes |
| **Thread-safe** | No (node-local) | No (agent-local) | ✅ Yes (with locks) |
| **Token Impact** | None | Direct impact | None (unless passed) |
| **Use Cases** | Per-node counters | Conversation continuity | Multi-agent coordination |

**When to use NODE STATE:**
- Track iteration count in loops
- Store user IDs, session IDs
- Maintain per-node application flags
- Count messages per node
- Store data NOT needed by LLM or other nodes

**When to use MEMORY:**
- Enable conversation continuity
- Provide context to LLM
- Reference previous messages
- Build on earlier discussion
- Maintain conversation flow

**When to use GRAPH STATE:**
- Coordinate between multiple agents in a workflow
- Track global counters across all nodes
- Share configuration across the graph
- Implement multi-agent voting or consensus
- Maintain workflow-level state
- Aggregate results from multiple agents

**GraphState Best Practices:**

```python
# Initialize graph with initial state
graph = Graph(
    start=my_node,
    initial_state={'counter': 0, 'results': []}
)

# In node's process() method:
async def process(self, context):
    # Always check if graph_state is available
    if context.graph_state:
        # Get value with default
        counter = await context.graph_state.get('counter', 0)
        
        # Update value
        await context.graph_state.set('counter', counter + 1)
        
        # Batch update
        await context.graph_state.update({'x': 1, 'y': 2})
        
        # Atomic transactions for read-modify-write
        async with context.graph_state.transaction() as state:
            state['x'] = state.get('x', 0) + 1
            state['y'] = state.get('y', 0) + 1
    
    return {'done': True}

# After graph execution, access final state
result = await graph.run()
final_counter = await graph.get_state('counter')
snapshot = graph.get_state_snapshot()
```

In [8]:
print("=== Example 7: Complete AgentConfig ===")
print()

# Define a simple tool
@tool
def get_current_time() -> str:
    """Get the current time."""
    return time.strftime('%H:%M:%S')

# Define hooks for monitoring
async def before_call_hook(msgs, context) -> None:
    print('before_call_hook payload:', msgs)
    print("  🔔 Hook: About to call LLM")

async def after_call_hook(result, context) -> None:
    print('after_call_hook payload:', result)
    print("  🔔 Hook: LLM call completed")

init_state = default_node_state()
init_state.update({'session_id': 'session_123', 'user_id': 'user_456'})
# Create comprehensive configuration
full_config = AgentConfig(
    # Core settings
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='advanced_assistant',
    description='A fully configured assistant with all features',
    
    # Prompting
    system_prompt='You are an advanced AI assistant with access to tools and context.',
    prompt_template='User: {{ query }}\nContext: {{ context }}',
    
    # Memory management
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=8
    ),
    
    # Context initialization
    preload_messages=[
        {'role': 'user', 'content': [{'text':'What can you help me with?'}]},
        {'role': 'assistant', 'content': [{'text':'I can help with questions, use tools, and maintain conversation context.'}]},
    ],
    
    # Output configuration
    output_mode='text',
    
    # Tools
    tools=[get_current_time],
    
    # Hooks and callbacks
    before_llm_hooks=[before_call_hook],
    after_llm_hooks=[after_call_hook],

    # Initial state
    initial_state=init_state,
)

# Create agent
advanced_agent = Agent(full_config)

print("🤖 Advanced Assistant (Full Configuration)")
print()
print("📋 Configuration Summary:")
print(f"  Name: {advanced_agent.config.name}")
print(f"  Description: {advanced_agent.config.description}")
print(f"  Memory Policy: {advanced_agent.config.memory_config.policy}")
print(f"  Memory Window: {advanced_agent.config.memory_config.window}")
print(f"  Preloaded Messages: {len(advanced_agent.config.preload_messages)}")
print(f"  Tools: {len(advanced_agent.config.tools)}")
print(f"  Hooks: {len(advanced_agent.config.before_llm_hooks)} before, {len(advanced_agent.config.after_llm_hooks)} after")
print(f"  Output Mode: {advanced_agent.config.output_mode}")
print()

# Test the agent
print("💬 Testing agent:")
query = "What time is it?"
print(f"User: {query}")
print()

result = await advanced_agent.do({
    'query': query,
    'context': 'Testing full configuration'
})

print()
print(f"Assistant: {result.content}")
print()

print("📊 Analysis:")
print("✓ All configuration options demonstrated")
print("✓ Hooks called before and after LLM")
print("✓ Tools available for use")
print("✓ Memory policy active")
print("✓ Initial state set")
print("✓ Prompt template applied")
print("✓ Ready for production use")

=== Example 7: Complete AgentConfig ===

🤖 Advanced Assistant (Full Configuration)

📋 Configuration Summary:
  Name: advanced_assistant
  Description: A fully configured assistant with all features
  Memory Policy: rolling_window
  Memory Window: 8
  Preloaded Messages: 2
  Tools: 1
  Hooks: 1 before, 2 after
  Output Mode: text

💬 Testing agent:
User: What time is it?

before_call_hook payload: [{'content': [{'text': 'What can you help me with?'}], 'role': 'user'}, {'content': [{'text': 'I can help with questions, use tools, and maintain conversation context.'}], 'role': 'assistant'}, {'role': 'user', 'content': [{'text': 'User: What time is it?\nContext: Testing full configuration'}]}]
  🔔 Hook: About to call LLM
tool_call: {'id': 'call_wLIoVeGuN8bBceP9jpJsH0rp', 'type': 'function', 'function': {'name': 'get_current_time', 'arguments': {}}}
after_call_hook payload: id=None type='text' content='Current time: 15:54:14 (server time). Would you like me to convert to another time zone or 

## 💪 Hands-On Exercises

Test your understanding with these practical exercises!

## Key Concepts Recap

### AgentConfig: Your Configuration Hub

```python
config = AgentConfig(
    model=...,              # Required: LLM model
    name=...,               # Agent identifier
    description=...,        # What the agent does
    system_prompt=...,      # System instructions
    prompt_template=...,    # Jinja2 formatting
    memory_config=...,      # Memory management
    preload_messages=...,   # Initial context
    output_mode=...,        # 'text' or 'json'
    output_schema=...,      # Pydantic schema
    tools=...,              # Available tools
    before_llm_hooks=...,   # Pre-LLM hooks
    after_llm_hooks=...,    # Post-LLM hooks
    initial_state=...,      # Starting node state
)
```

### Memory Policies Comparison

| Policy | Best For | Token Impact | Context Retention |
|--------|----------|--------------|-------------------|
| **NULL** | FAQ bots, stateless APIs | Minimal | None |
| **ROLLING_WINDOW** | Chat bots, support agents | Bounded | Recent messages |
| **CUSTOM** | Complex requirements | Variable | Fully customizable |

### State vs Memory vs GraphState (Quick Reference)

```python
# NODE STATE: Per-node tracking (NOT sent to LLM)
context.state['message_count'] = 5
context.state['user_id'] = 'user_123'

# MEMORY: Conversation history (SENT to LLM)
# Managed automatically by MemoryManager
agent.config.memory_config = MemoryConfig(
    policy=MemoryPolicyType.ROLLING_WINDOW,
    window=10
)

# GRAPH STATE: Shared across all nodes (NOT sent to LLM)
# Available when agent is part of a graph
if context.graph_state:
    await context.graph_state.set('total_messages', 100)
    counter = await context.graph_state.get('workflow_step', 0)
```

### When to Use Each Memory Policy

**Use NULL when:**
- Processing independent requests
- Building stateless APIs
- Token efficiency is critical
- No conversation continuity needed

**Use ROLLING_WINDOW when:**
- Building conversational agents
- Need recent context
- Want predictable token usage
- Typical chat bot scenarios

**Use CUSTOM when:**
- Need importance-based retention
- Implementing summarization
- Topic-based memory filtering
- Complex memory requirements

### Best Practices Summary

1. **Always use AgentConfig** for new agents (cleaner, more maintainable)
2. **Choose memory policy** based on use case, not default
3. **Monitor token usage** and adjust window size accordingly
4. **Use state for tracking**, memory for conversation, **GraphState for coordination**
5. **Preload important context** to save tokens and setup time
6. **Test memory behavior** with realistic conversation lengths
7. **Document memory decisions** in your code
8. **Consider custom policies** for advanced use cases
9. **Use GraphState** when agents need to coordinate in multi-agent workflows
10. **Always check** if `context.graph_state` is available before using it

In [ ]:
# Exercise 1: Your code here

# TODO: Define company policy preload messages
# TODO: Create AgentConfig with rolling window (6 messages)
# TODO: Create agent that tracks question count in state
# TODO: Test with a conversation

### Exercise 2: Create a Stateless Q&A Agent

Build a FAQ agent that:
- Uses NULL memory policy
- Has a clear system prompt about being stateless
- Includes preloaded FAQ examples
- Processes each question independently

In [ ]:
# Exercise 2: Your code here

# TODO: Create AgentConfig with NULL policy
# TODO: Preload FAQ examples
# TODO: Test with follow-up questions to verify statelessness

### Exercise 3: Implement a Custom Memory Policy

Create a research assistant with custom memory that:
- Keeps all messages tagged with 'research' metadata
- Keeps the last 3 regular messages
- Drops all other messages
- Test with a mix of research and casual messages

In [ ]:
# Exercise 3: Your code here

# TODO: Define custom memory function
# TODO: Create AgentConfig with CUSTOM policy
# TODO: Test with tagged messages

### Exercise 4: Memory Policy Comparison

Create three versions of the same agent with different memory policies:
- Version A: NULL policy
- Version B: ROLLING_WINDOW with window=4
- Version C: ROLLING_WINDOW with window=10

Run the same conversation through all three and compare the results.

In [ ]:
# Exercise 4: Your code here

# TODO: Create three agent configurations
# TODO: Define a test conversation
# TODO: Run conversation through all three agents
# TODO: Compare and analyze the differences

## ✅ Solutions

Try the exercises yourself first! Solutions are provided below.

In [9]:
# Solution 1: Limited-Memory Support Agent

# Company policies to preload
company_policies = [
    {'role': 'user', 'content': [{'text':'What is your return policy?'}]},
    {'role': 'assistant', 'content': [{'text':'We offer 30-day returns for all products with receipt.'}]},
    {'role': 'user', 'content': [{'text':'What are your business hours?'}]},
    {'role': 'assistant', 'content': [{'text':'We are open Monday-Friday 9am-6pm EST.'}]},
]

class SupportAgent(Agent):
    """Support agent that tracks questions."""
    
    async def process(self, context: ExecutionContext):
        # Track question count in state
        question_count = context.state.get('question_count', 0) + 1
        context.state['question_count'] = question_count
        
        print(f"📊 Question #{question_count}")
        
        result = await super().process(context)
        return result

# Create configuration
support_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='support_agent',
    system_prompt='You are a friendly customer support agent. Help users with their questions about our products and policies.',
    preload_messages=company_policies,
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=6
    ),
    initial_state=default_node_state(question_count=0)
)

support_agent = SupportAgent(config=support_config)

print("=== Solution 1: Limited-Memory Support Agent ===")
print()

# Test conversation
questions = [
    "Can I return a product I bought 20 days ago?",
    "Are you open on weekends?",
    "What about the return policy again?",
]

for q in questions:
    print(f"User: {q}")
    # Use structured content format to match preloaded messages
    result = await support_agent.do({'messages': [{'role': 'user', 'content': [{'text': q}]}]})
    print(f"Assistant: {result.content}")
    print()

print(f"✅ Total questions asked: {support_agent._state['question_count']}")

=== Solution 1: Limited-Memory Support Agent ===

User: Can I return a product I bought 20 days ago?
📊 Question #1
Assistant: Yes. You’re within our 30-day return window as long as you have the receipt. If you’d like, I can help start the return—could you share your order number or the email used for the purchase? Also tell me whether you’d prefer a refund or an exchange.

User: Are you open on weekends?
📊 Question #2
########################################################
error: content_type=<Y> | unsupported type
########################################################
Assistant: {'error': 'content_type=<Y> | unsupported type'}

User: What about the return policy again?
📊 Question #3
########################################################
error: content_type=<Y> | unsupported type
########################################################
Assistant: {'error': 'content_type=<Y> | unsupported type'}

✅ Total questions asked: 3


In [9]:
# Solution 1: Limited-Memory Support Agent

# Company policies to preload
company_policies = [
    {'role': 'user', 'content': [{'text':'What is your return policy?'}]},
    {'role': 'assistant', 'content': [{'text':'We offer 30-day returns for all products with receipt.'}]},
    {'role': 'user', 'content': [{'text':'What are your business hours?'}]},
    {'role': 'assistant', 'content': [{'text':'We are open Monday-Friday 9am-6pm EST.'}]},
]

class SupportAgent(Agent):
    """Support agent that tracks questions."""
    
    async def process(self, context: ExecutionContext):
        # Track question count in state
        question_count = context.state.get('question_count', 0) + 1
        context.state['question_count'] = question_count
        
        print(f"📊 Question #{question_count}")
        
        result = await super().process(context)
        return result

# Create configuration
support_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='support_agent',
    system_prompt='You are a friendly customer support agent. Help users with their questions about our products and policies.',
    preload_messages=company_policies,
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=6
    ),
    initial_state=default_node_state(question_count=0)
)

support_agent = SupportAgent(config=support_config)

print("=== Solution 1: Limited-Memory Support Agent ===")
print()

# Test conversation
questions = [
    "Can I return a product I bought 20 days ago?",
    "Are you open on weekends?",
    "What about the return policy again?",
]

for q in questions:
    print(f"User: {q}")
    # Use structured content format to match preloaded messages
    result = await support_agent.do({'messages': [{'role': 'user', 'content': [{'text': q}]}]})
    print(f"Assistant: {result.content}")
    print()

print(f"✅ Total questions asked: {support_agent._state['question_count']}")

=== Solution 1: Limited-Memory Support Agent ===

User: Can I return a product I bought 20 days ago?
📊 Question #1
Assistant: Yes. You’re within our 30-day return window as long as you have the receipt. If you’d like, I can help start the return—could you share your order number or the email used for the purchase? Also tell me whether you’d prefer a refund or an exchange.

User: Are you open on weekends?
📊 Question #2
########################################################
error: content_type=<Y> | unsupported type
########################################################
Assistant: {'error': 'content_type=<Y> | unsupported type'}

User: What about the return policy again?
📊 Question #3
########################################################
error: content_type=<Y> | unsupported type
########################################################
Assistant: {'error': 'content_type=<Y> | unsupported type'}

✅ Total questions asked: 3


In [10]:
# Solution 2: Stateless FAQ Agent

# FAQ examples to preload
faq_examples = [
    {'role': 'user', 'content': [{'text':'How do I reset my password?'}]},
    {'role': 'assistant', 'content': [{'text':'Click "Forgot Password" on the login page and follow the email instructions.'}]},
    {'role': 'user', 'content': [{'text':'Where can I find my order history?'}]},
    {'role': 'assistant', 'content': [{'text':'Go to Account > Orders to view your order history.'}]},
]

# Create stateless FAQ agent
faq_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='faq_bot',
    system_prompt="""You are a FAQ bot that answers questions independently.
You do NOT remember previous questions. Each question is answered in isolation.
If asked about previous questions, clearly state that you don't have memory.""",
    preload_messages=faq_examples,
    memory_config=MemoryConfig(policy=MemoryPolicyType.NULL),
)

faq_agent = Agent(faq_config)

print("=== Solution 2: Stateless FAQ Agent ===")
print()

# Test with follow-up questions
test_questions = [
    "How do I contact support?",
    "What about billing questions?",
    "What was my first question?",  # Should NOT remember
]

for i, q in enumerate(test_questions, 1):
    print(f"Q{i}: {q}")
    # Use structured content format to match preloaded messages
    result = await faq_agent.do({'messages': [{'role': 'user', 'content': [{'text': q}]}]})
    print(f"A{i}: {result.content}")
    print()

print("✅ Agent correctly doesn't remember previous questions (NULL policy)")

=== Solution 2: Stateless FAQ Agent ===

Q1: How do I contact support?
A1: I don’t have memory of past questions, but I can help you contact support. Here are general ways to reach them and how to speed things up:

Common contact options
- Website: Look for a “Contact Us” or “Support” link (usually in the footer).
- Phone: Most companies list a support phone number with hours.
- Email: Support or help@ or similar email address.
- Live chat: Often available on the support page or app.
- Help center / ticket system: Submit a support ticket with details of your issue.
- Social media: Some brands respond via Twitter/X or Facebook messages.
- In-app support: If you’re using a app, check Help > Contact Support or Support/Feedback.

What to have ready
- Your account or order number (if applicable)
- A clear description of the issue
- Steps to reproduce the problem
- Your device/OS and app version (if it’s a mobile app)
- Screenshots or screen recordings (optional but helpful)
- Any relevant e

In [5]:
# Solution 3: Custom Memory Policy for Research Assistant

def research_memory_policy(messages: list[dict[str, Any]]) -> list[dict[str, Any]]:
    """
    Keep all 'research' tagged messages + last 3 regular messages.
    """
    research_messages = [
        m for m in messages 
        if m.get('metadata', {}).get('type') == 'research'
    ]
    
    regular_messages = [
        m for m in messages 
        if m.get('metadata', {}).get('type') != 'research'
    ]
    
    recent_regular = regular_messages[-3:]
    
    result = research_messages + recent_regular
    
    print(f"  📊 Memory: {len(messages)} → {len(result)} messages")
    print(f"     Research: {len(research_messages)}, Recent: {len(recent_regular)}")
    
    return result

# Create research assistant
research_config = AgentConfig(
    model=EchoModel(),
    name='research_assistant',
    system_prompt='You are a research assistant. Prioritize research-related information.',
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.CUSTOM,
        callable=research_memory_policy
    ),
)

research_agent = Agent(research_config)

print("=== Solution 3: Custom Memory Research Assistant ===")
print()

# Simulate messages with metadata
test_messages = [
    {'role': 'user', 'content': [{'text':'Research topic: Quantum computing'}], 'metadata': {'type': 'research'}},
    {'role': 'assistant', 'content': [{'text':'Noted: Quantum computing research'}]},
    {'role': 'user', 'content': [{'text':'What\'s the weather?'}], 'metadata': {'type': 'casual'}},
    {'role': 'assistant', 'content': [{'text':'Sunny today'}]},
    {'role': 'user', 'content': [{'text':'Research topic: Machine learning'}], 'metadata': {'type': 'research'}},
    {'role': 'assistant', 'content': [{'text':'Noted: ML research'}]},
    {'role': 'user', 'content': [{'text':'Tell me a joke'}], 'metadata': {'type': 'casual'}},
    {'role': 'assistant', 'content': [{'text':'Why did the...'}]},
]

print("Testing custom memory filtering:")
filtered = research_memory_policy(test_messages)
print()

print("Retained messages:")
for msg in filtered:
    msg_type = msg.get('metadata', {}).get('type', 'unknown')
    marker = "🔬" if msg_type == 'research' else "💬"
    print(f"  {marker} {msg['role']}: {msg['content'][:40]}...")

print()
print("✅ Custom policy retains all research messages + recent regular messages")

=== Solution 3: Custom Memory Research Assistant ===

Testing custom memory filtering:
  📊 Memory: 8 → 5 messages
     Research: 2, Recent: 3

Retained messages:
  🔬 user: [{'text': 'Research topic: Quantum computing'}]...
  🔬 user: [{'text': 'Research topic: Machine learning'}]...
  💬 assistant: [{'text': 'Noted: ML research'}]...
  💬 user: [{'text': 'Tell me a joke'}]...
  💬 assistant: [{'text': 'Why did the...'}]...

✅ Custom policy retains all research messages + recent regular messages


In [ ]:
# Solution 4: Memory Policy Comparison

print("=== Solution 4: Memory Policy Comparison ===")
print()

# Create three agents with different memory policies
agent_null = Agent(AgentConfig(
    model=EchoModel(),
    name='agent_null',
    system_prompt='You are an assistant.',
    memory_config=MemoryConfig(policy=MemoryPolicyType.NULL),
))

agent_small_window = Agent(AgentConfig(
    model=EchoModel(),
    name='agent_small',
    system_prompt='You are an assistant.',
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=4
    ),
))

agent_large_window = Agent(AgentConfig(
    model=EchoModel(),
    name='agent_large',
    system_prompt='You are an assistant.',
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=10
    ),
))

# Test conversation
conversation = [
    "My name is Bob.",
    "I live in New York.",
    "I work as a software engineer.",
    "I like pizza.",
    "What's my name?",
    "Where do I live?",
    "What do I do for work?",
]

agents = [
    ('NULL Policy', agent_null),
    ('Rolling Window (4)', agent_small_window),
    ('Rolling Window (10)', agent_large_window),
]

for agent_name, agent in agents:
    print(f"\n{'='*60}")
    print(f"Testing: {agent_name}")
    print(f"Memory: {agent.config.memory_config.policy}")
    if agent.config.memory_config.policy == MemoryPolicyType.ROLLING_WINDOW:
        print(f"Window: {agent.config.memory_config.window}")
    print('='*60)
    
    for msg in conversation:
        # Use structured content format
        result = await agent.do({'messages': [{'role': 'user', 'content': [{'text': msg}]}]})
        if '?' in msg:
            print(f"Q: {msg}")
            print(f"A: {result.content}")

print()
print("📊 Comparison Analysis:")
print("  NULL: Cannot answer any reference questions (no memory)")
print("  Small Window (4): Remembers recent info, may forget earlier details")
print("  Large Window (10): Remembers all conversation details")
print()
print("✅ Different memory policies suit different use cases!")

## 🎯 Summary & Next Steps

### Congratulations! 🎉

You've mastered agent configuration and memory management in Spark! You now understand how to configure agents for different use cases, manage conversation memory effectively, and coordinate multiple agents using GraphState.

### What You Learned:

✅ **AgentConfig Mastery**
- Centralized configuration with AgentConfig
- Type-safe configuration with Pydantic
- All configuration options and their purposes
- Benefits over scattered configuration

✅ **Memory Management**
- Three memory policies: NULL, ROLLING_WINDOW, CUSTOM
- When to use each policy
- How to configure memory windows
- Custom memory policies for advanced use cases

✅ **State vs Memory vs GraphState**
- Critical distinctions between node state, memory, and graph state
- When to use each mechanism
- How they affect LLM behavior and token usage
- Best practices for all three

✅ **Advanced Patterns**
- Preloading messages for context initialization
- Custom memory policies for complex requirements
- Using GraphState for multi-agent coordination
- Thread-safe state sharing across nodes

### Key Patterns to Remember:

```python
# Stateless agent
config = AgentConfig(
    model=model,
    memory_config=MemoryConfig(policy=MemoryPolicyType.NULL)
)

# Conversational agent
config = AgentConfig(
    model=model,
    memory_config=MemoryConfig(
        policy=MemoryPolicyType.ROLLING_WINDOW,
        window=10
    )
)

# Multi-agent workflow with GraphState
graph = Graph(
    start=agent1,
    initial_state={'counter': 0, 'results': []}
)

# In agents' process() method
async def process(self, context):
    # Node state (local to this agent)
    context.state['my_count'] = 1
    
    # Graph state (shared across all agents)
    if context.graph_state:
        total = await context.graph_state.get('counter', 0)
        await context.graph_state.set('counter', total + 1)
    
    return {'done': True}
```

### 📚 Related Resources:

- Example files: `e006_agent_with_config.py`, `e009_tool_agent.py`
- Source: `spark/agents/config.py`, `spark/agents/memory.py`
- Tutorial 4: Your First AI Agent (review if needed)
- Tutorial 6: LLM-Powered Routing (builds on this)
- CLAUDE.md: GraphState documentation in Core Architecture section

### 🚀 Next Tutorial: Tools - Extending Agent Capabilities

In **Tutorial 8**, you'll learn how to:
- Create custom tools for agents
- Understand tool calling mechanics
- Configure tool choice strategies
- Debug tool execution
- Build multi-tool agents
- Implement complex tool-based workflows

### 🔧 Before You Move On:

Make sure you can:
1. ✅ Create agents with AgentConfig
2. ✅ Choose and configure appropriate memory policies
3. ✅ Distinguish between state, memory, and GraphState
4. ✅ Use preloaded messages effectively
5. ✅ Implement custom memory policies
6. ✅ Use GraphState to coordinate multiple agents in a graph

### 🎓 Tutorial Series Progress:

- ✅ **Tutorial 1: Hello Spark** - Basic nodes
- ✅ **Tutorial 2: Batch Processing** - Parallel execution
- ✅ **Tutorial 3: Simple Flows** - Graph basics
- ✅ **Tutorial 4: Your First AI Agent** - Agent fundamentals
- ✅ **Tutorial 5: Conditional Routing** - Decision making
- ✅ **Tutorial 6: Agent Config & Memory** - *You are here!* 🎯
- ➡️ **Tutorial 7: Tools** - Extending agents

### 🌟 Pro Tips:

- **Start with ROLLING_WINDOW** for most conversational agents
- **Monitor token usage** to optimize window size
- **Use NULL** for stateless, high-performance APIs
- **Consider CUSTOM** when standard policies don't fit
- **Use node state** for per-agent tracking
- **Use memory** for LLM conversation context
- **Use GraphState** for multi-agent coordination
- **Preload context** to save setup time and tokens
- **Test memory behavior** with realistic conversations
- **Document your decisions** about state/memory/GraphState
- **Always check** `if context.graph_state` before using it

### 🎯 Challenge Before Next Tutorial:

Build a "Multi-Agent Workflow" that:
1. Has two agents: Classifier and Processor
2. Classifier uses NULL memory (stateless)
3. Processor uses rolling window memory (window=6)
4. Both agents update a shared counter in GraphState
5. GraphState tracks total classifications and total processing time
6. Each agent tracks its own invocation count in node state

This will prepare you for adding tools in Tutorial 8!

---

**You're now ready to build production-ready multi-agent workflows with proper configuration, memory management, and state coordination!** 🚀

Have questions or feedback? Check the Spark documentation or open an issue on GitHub.